In [1]:
from pathlib import Path
import json
import requests
from requests.auth import HTTPBasicAuth
from watson_developer_cloud import PersonalityInsightsV3

# 本のデータ名
books = ["bocchan.txt", "chumonno_oi_ryoriten.txt", "hashire_merosu.txt", "kaijin_nijumenso.txt", "ningen_shikkaku.txt"]
# 抽出した本のストーリーを保存するフォルダ
stored_path = Path("./extracted_bookdata")

def get_personality(text):
    """
    Personality Insightsを用いた性格分析の結果を取得する
    参考サイト：http://tmngtm.hatenablog.com/entry/2016/10/14/203901
    -> return responseで400コードが出てうまくいかず(後で分かったことだが、恐らくContent-Languageをenにしていた...)
    -> 公式マニュアルを読み解いた
    ※事前に、pip install --upgrade watson-developer-cloud
    """
    api_version = "2017-10-13"
    api_username = "396677bb-f0d3-4650-9d16-9e479f9e1f78"
    api_password = "CyJVefXvDdIA"
    api_url = "https://gateway.watsonplatform.net/personality-insights/api"
    
    personality_insights = PersonalityInsightsV3(
        version = api_version,
        username = api_username,
        password = api_password,
        url = api_url
    )
    
    profile = personality_insights.profile(
        content = text,
        content_type = "text/plain",
        accept = "application/json",
        content_language = "ja",
        accept_language = "en",
        raw_scores = True,
    )
    
    return profile

if __name__ == "__main__":
    for book in books:
        book_path = Path(stored_path.name + "/" + book) # Path(Path("./extracted_bookdata") / book) でもOK
        # 入力となる本のデータ
        story = book_path.read_text(encoding="utf-8")
        # personality insightsに本のデータを投げる
        story_personality = get_personality(story)
        # 辞書からJSON形式に変換する
        # 参考サイト：https://tmg0525.hatenadiary.jp/entry/2018/03/30/204448
        print(json.dumps(story_personality, indent = 4))
        break

{
    "word_count": 73303,
    "processed_language": "ja",
    "personality": [
        {
            "trait_id": "big5_openness",
            "name": "Openness",
            "category": "personality",
            "percentile": 0.8585328275429231,
            "raw_score": 0.6386292309542972,
            "significant": true,
            "children": [
                {
                    "trait_id": "facet_adventurousness",
                    "name": "Adventurousness",
                    "category": "personality",
                    "percentile": 0.3940140763269059,
                    "raw_score": 0.5398013194288398,
                    "significant": true
                },
                {
                    "trait_id": "facet_artistic_interests",
                    "name": "Artistic interests",
                    "category": "personality",
                    "percentile": 0.8655147125718997,
                    "raw_score": 0.7270973328827577,
                    "significan

In [ ]:
def get_personality_old(text):
    """
    上記参考サイトのコード＋少々改変版。
    Content-Languageをjaにしていればできてたっぽい。
    """
    # endpoint url
    api_url = "https://gateway.watsonplatform.net/personality-insights/api/v2/profile"
    # username and password for this api
    api_username = "396677bb-f0d3-4650-9d16-9e479f9e1f78"
    api_password = "CyJVefXvDdIA"
    
    # set query
    query = {
        "include_raw": "false",
        "header": "false"
    }
    
    # set header
    headers = {
        "Content-Type": "text/plain", # 入力テキストの形式。今回はstringのテキストを渡す設定。
        "Content-Language": "en",     # 入力言語モードの指定。jaとすると日本語の入力を受け付ける。
        "Accept": "application/json", # 応答ファイルの形式。今回はjson形式で受け取る。
        "Accept-Language": "en",      # 処理言語モードの設定。jaとすると日本語として処理する。
        "include_raw": "false"
    }
    
    # set body
    body = text
    body = body.encode("utf-8") # 参考サイトのコードに追記
    
    # post
    response = requests.post(
        api_url,
        auth = HTTPBasicAuth(api_username, api_password),
        params = query,
        headers = headers,
        data = body
    )
    return response